In [ ]:
# ============ IMPORTS ============
from openai import OpenAI
import pandas as pd
import json
import os
import re


# ============ 1. API KEY & CLIENT SETUP ============
os.environ["OPENAI_API_KEY"] = "your_api_keys_here"  # Replace locally if needed
api_key = os.getenv("OPENAI_API_KEY")
if api_key is None:
    raise ValueError("ERROR: OPENAI_API_KEY environment variable not set.")

client = OpenAI(api_key=api_key)


# ============ 2. LOAD INPUT DATA ============
df = pd.read_excel("right_to_exit_override_references_rich_excerpt.xlsx")
print("Loaded columns:", df.columns.tolist())  # Diagnostic


# ============ 3. DEFINE SCORING WEIGHTS ============
weights = {
    "visibility_accessibility": 0.25,
    "clarity_labels": 0.20,
    "effectiveness": 0.30,
    "friction": 0.15,
    "regulatory_alignment": 0.10
}

results = []


# ============ 4. LOOP THROUGH EVIDENCE ROWS ============
for _, row in df.iterrows():
    prompt = f"""
You must score the following evidence excerpt based on these 5 criteria (each score 0–5):

1. Visibility & Accessibility – How easy it is to locate exit/override settings.
2. Clarity of Labels – Whether descriptions are intuitive and consequences clearly explained.
3. Effectiveness of Mechanism – Whether the action fully works (e.g., deletion honored, consent updated).
4. Friction & Dark Patterns – Whether users face obstacles or discouragement during exit/override.
5. Regulatory Compliance – Alignment with legal privacy rights (e.g., GDPR).

Use these weights (sum = 1):
- Visibility & Accessibility: 0.25
- Clarity of Labels: 0.20
- Effectiveness of Mechanism: 0.30
- Friction & Dark Patterns: 0.15
- Regulatory Compliance: 0.10

Final score = sum(each_score × weight).

Evidence:
- Source: {row['Source']}
- Excerpt: {row['Key Excerpt (English)']}

⚠ OUTPUT REQUIREMENTS:
Return ONLY VALID JSON (no markdown, no code block, no extra text).

Format:
{{
  "scores": {{
    "visibility_accessibility": X,
    "clarity_labels": X,
    "effectiveness": X,
    "friction": X,
    "regulatory_alignment": X
  }},
  "weighted_final_score": X,
  "justification": "short justification within 70 words"
}}
"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": "You are an ethics evaluation expert scoring right-to-exit or override UX."
            },
            {"role": "user", "content": prompt}
        ]
    )

    raw_content = response.choices[0].message.content.strip()

    # ===== Robust JSON parsing =====
    try:
        result_json = json.loads(raw_content)
    except json.JSONDecodeError:
        match = re.search(r"\{[\s\S]*\}", raw_content)
        if match:
            try:
                result_json = json.loads(match.group())
            except json.JSONDecodeError:
                print("⚠ JSON parsing failed.\nRaw output:")
                print(raw_content)
                continue
        else:
            print("⚠ No JSON detected in response.\nRaw output:")
            print(raw_content)
            continue

    # Attach metadata
    result_json["Source"] = row["Source"]
    result_json["URL"] = row["URL"]

    # Retrieve raw scores
    s = result_json["scores"]

    # Compute weighted scores (keep 0–5 scale)
    detailed_weighted_scores = {
        key: round(s[key] * weights[key], 2) for key in s
    }

    weighted_score = round(sum(detailed_weighted_scores.values()), 2)

    result_json["weighted_scores_detailed"] = detailed_weighted_scores
    result_json["weighted_final_score"] = weighted_score

    results.append(result_json)



# ============ 5. EXPORT RESULTS ============
with open("right_to_exit_scoring_weighted.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

print("🎉 Scoring successfully completed using GPT-4o.")
print("📁 Output saved: right_to_exit_scoring_weighted.json")


Loaded columns: ['Source', 'Key Excerpt (English)', 'Relevance to Exit/Override UX', 'Citation Type', 'URL']
🎉 Scoring successfully completed using GPT-4o.
📁 Output saved: right_to_exit_scoring_weighted.json
